In [1]:
!pip install transformers
!pip install datasets
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 4.5 MB/s eta 0:00:00


In [2]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import BertForSequenceClassification, Trainer, TrainingArguments, TrainerCallback
from datasets import Dataset, load_dataset

In [3]:
import os

model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=1)


dataset = load_dataset("md_gender_bias", name="convai2_inferred")

# Access the train, validation, and test splits
train_data = Dataset.from_dict(dataset["train"][0:5500])
validation_data = dataset["validation"]
test_data = dataset["test"]

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Generating train split:   0%|          | 0/131438 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/7801 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7801 [00:00<?, ? examples/s]

Run the following cell to train a model. Alter the index number at

learning_rate = hyperparameters['lr'][0],
weight_decay = hyperparameters['wd'][0]

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import BertForSequenceClassification, Trainer, TrainingArguments, TrainerCallback
from datasets import Dataset, load_dataset


import os

model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=1)


dataset = load_dataset("md_gender_bias", name="convai2_inferred")

# Access the train, validation, and test splits
train_data = Dataset.from_dict(dataset["train"][0:5500])
validation_data = dataset["validation"]
test_data = dataset["test"]
hyperparameters = {'lr':[1e-5, 5e-5, 1e-4], 'wd':[0, 1e-3, 5e-3]}

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

def process_dataset(dataset):
    new_dataset_dict = {
        'text': [entry['text'] for entry in dataset],
        'labels': [entry['binary_score'] if entry['binary_label'] == 1 else 1 - entry['binary_score'] for entry in dataset]
    }
    new_dataset = Dataset.from_dict(new_dataset_dict)
    return new_dataset

training_args = TrainingArguments(
    output_dir='./output',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    save_total_limit=2,
    evaluation_strategy="epoch",

    learning_rate = hyperparameters['lr'][0],
    weight_decay = hyperparameters['wd'][0]
)

processed_train_dataset = process_dataset(train_data)
processed_validation_dataset = process_dataset(validation_data)
tokenized_train_datasets = processed_train_dataset.map(tokenize_function, batched=True)
tokenized_validation_datasets = processed_validation_dataset.map(tokenize_function, batched=True)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_datasets,
    eval_dataset=tokenized_validation_datasets,
    tokenizer=tokenizer,
    compute_metrics=None,
    )

class CustomCallback(TrainerCallback):
    def __init__(self):
        self.best_validation_loss = float('inf')

    def on_epoch_end(self, args, state, control, model, tokenizer, **kwargs):
        # Save the model at the end of each epoch
        model.save_pretrained(f'model_saved_epoch_{state.epoch}')

        # Compute and log training loss
        train_loss = state.log_history[-1].get("loss", None)
        if train_loss is not None:
            print(f"Epoch {state.epoch}: Training Loss: {train_loss:.4f}")

        # Evaluate the model on the validation dataset and compute validation loss
        eval_results = trainer.evaluate()
        print(f"Epoch {state.epoch}")
        print(eval_results)


# Initialize the custom callback
custom_callback = CustomCallback()

# Add the callback to the trainer
trainer.add_callback(custom_callback)

# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=tokenized_datasets,  # Use combined policy and non-policy inputs
# )

trainer.train()

model.save_pretrained('model_saved_00')

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

...

Map:   0%|          | 0/5500 [00:00<?, ? examples/s]

Map:   0%|          | 0/7801 [00:00<?, ? examples/s]

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
1,0.031000,0.006320
2,0.009000,0.004770
3,0.004500,0.004396


Epoch 1.0: Training Loss: 0.0310
Epoch 1.0
{'eval_loss': 0.0063200099393725395}
Epoch 2.0: Training Loss: 0.0090
Epoch 2.0
{'eval_loss': 0.004770294763147831}
Epoch 3.0: Training Loss: 0.0045
Epoch 3.0
{'eval_loss': 0.004396312404423952}


testing. Notice that at the line `model_path = 'model_output'`, modify the string to your model path

In [5]:
from transformers import BertForSequenceClassification, BertTokenizer, Trainer, TrainingArguments
from datasets import load_dataset

# Load your test dataset
test_dataset = load_dataset("md_gender_bias", name="convai2_inferred")["test"]

# Replace 'model_output' with the actual path where you saved your model
model_path = 'model_output'

# Load the pre-trained model and tokenizer
loaded_model = BertForSequenceClassification.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased", max_length=1024)

def process_dataset(dataset):
    new_dataset_dict = {
        'text': [entry['text'] for entry in dataset],
        'labels': [entry['binary_score'] if entry['binary_label'] == 1 else 1 - entry['binary_score'] for entry in dataset]
    }
    new_dataset = Dataset.from_dict(new_dataset_dict)
    return new_dataset

# Tokenize the test dataset
test_dataset = process_dataset(test_dataset)
tokenized_test_dataset = test_dataset.map(lambda examples: tokenizer(examples["text"], padding="max_length", truncation=True), batched=True)



# Set up Trainer with custom data collator
trainer = Trainer(
    model=loaded_model,
)

# Make predictions on the test dataset
predictions = trainer.predict(tokenized_test_dataset)

# Extract the predicted values
predicted_values = predictions.predictions.squeeze().tolist()

# Print the first few predicted values as an example
print(predicted_values[:5])


Map:   0%|          | 0/7801 [00:00<?, ? examples/s]

[0.5466815829277039, 0.5659842491149902, 0.5973338484764099, 0.648098886013031, 0.6818491816520691]


In [6]:
print(predicted_values)

[0.5466815829277039, 0.5659842491149902, 0.5973338484764099, 0.648098886013031, 0.6818491816520691, 0.5922430753707886, 0.17233702540397644, 0.6590470671653748, 0.5210486054420471, 0.6074684858322144, 0.5338666439056396, 0.8435412645339966, 0.6577248573303223, 0.6165161728858948, 0.20780512690544128, 0.556465208530426, 0.3397764563560486, 0.47799983620643616, 0.9699114561080933, 0.6307814717292786, 0.41468727588653564, 0.24886047840118408, 0.5284580588340759, 0.5609793066978455, 0.7129021883010864, 0.3834431767463684, 0.9894803762435913, 0.49089735746383667, 0.4269016683101654, 0.20432965457439423, 0.679970383644104, 0.5594537258148193, 0.4973469078540802, 0.5705516934394836, 0.586331844329834, 0.5329147577285767, 0.4232586622238159, 0.5756500363349915, 0.5100022554397583, 0.36927303671836853, 0.3663078248500824, 0.5662359595298767, 0.2231348752975464, 0.27828744053840637, 0.5895679593086243, 0.384768545627594, 0.44976431131362915, 0.4238963723182678, 0.4919602572917938, 0.611187934875

In [8]:
true_value = [data['labels'] for data in test_dataset]

In [9]:
print(true_value)

[0.5015000104904175, 0.5889000296592712, 0.551800012588501, 0.48909997940063477, 0.6385999917984009, 0.5605000257492065, 0.15679997205734253, 0.6536999940872192, 0.5406000018119812, 0.609499990940094, 0.5641999840736389, 0.7674000263214111, 0.6898999810218811, 0.6241999864578247, 0.45649999380111694, 0.5597000122070312, 0.3345000147819519, 0.4334999918937683, 0.9426000118255615, 0.5806999802589417, 0.3353000283241272, 0.13779997825622559, 0.5109999775886536, 0.5723999738693237, 0.6233999729156494, 0.33469998836517334, 0.9550999999046326, 0.44679999351501465, 0.3644000291824341, 0.21670001745224, 0.6263999938964844, 0.5648999810218811, 0.5436999797821045, 0.650600016117096, 0.5436999797821045, 0.6166999936103821, 0.4562000036239624, 0.5317999720573425, 0.6589999794960022, 0.3568999767303467, 0.3252999782562256, 0.6358000040054321, 0.20579999685287476, 0.19040000438690186, 0.6758000254631042, 0.366599977016449, 0.429099977016449, 0.45980000495910645, 0.44770002365112305, 0.55000001192092

In [10]:
correct_times = 0
for i in range(7801):
  if predicted_values[i]<0.5 and true_value[i]<0.5:
    correct_times += 1
  elif predicted_values[i]>=0.5 and true_value[i]>=0.5:
    correct_times += 1
print(correct_times)

6861


In [11]:
len(true_value)

7801

write prediction result to txt

In [12]:
with open("pred.txt", "w") as fp_pred:
  for val in predicted_values:
    fp_pred.write(str(val))
    fp_pred.write('\n')
with open("label.txt", "w") as fp_label:
  for val in true_value:
    fp_label.write(str(val))
    fp_label.write('\n')

Load BUG dataset

In [14]:
import pandas as pd
from datasets import Dataset

# Load the CSV file into a pandas DataFrame
df = pd.read_csv("gold_BUG.csv")

# Select specific columns from the DataFrame
selected_columns = ["sentence_text", "predicted gender"]
df = df[selected_columns]

# Convert the pandas DataFrame to a Hugging Face Dataset
gold_dataset = Dataset.from_pandas(df)



In [15]:
def process_gold_dataset(dataset):
    new_dataset_dict = {
        'text': [entry['sentence_text'] for entry in dataset],
        'labels': [1 if entry['predicted gender'] == "Male" else 0 for entry in dataset]
    }
    new_dataset = Dataset.from_dict(new_dataset_dict)
    return new_dataset

In [16]:
processed_gold_dataset = process_gold_dataset(gold_dataset)

In [19]:
tokenized_gold_dataset = processed_gold_dataset.map(lambda examples: tokenizer(examples["text"], padding="max_length", truncation=True), batched=True)



# Set up Trainer with custom data collator
trainer = Trainer(
    model=loaded_model,
)

# Make predictions on the test dataset
predictions = trainer.predict(tokenized_gold_dataset)

# Extract the predicted values
predicted_values = predictions.predictions.squeeze().tolist()

# Print the first few predicted values as an example
print(predicted_values[:5])


Map:   0%|          | 0/1717 [00:00<?, ? examples/s]

[0.8678800463676453, 0.6311075091362, 0.25459346175193787, 0.5640638470649719, 0.7390254139900208]


In [22]:
true_value = processed_gold_dataset["labels"]
with open("pred_gold.txt", "w") as fp_pred:
  for val in predicted_values:
    fp_pred.write(str(val))
    fp_pred.write('\n')
with open("label_gold.txt", "w") as fp_label:
  for val in true_value:
    fp_label.write(str(val))
    fp_label.write('\n')